# Module 4 Project - Time-Series Analysis Using Zillow Data



# Objectives

The goal of this project is to provide consulting for a fictional real-estate investment firm. The question we will answer in conlusion of this research is 

> *What are the top 5 best zipcodes for the firm to invest in?* 

The data we will be working with comes from the Zillow Research Page. Specifically, we are going to look at zillow_data.csv. 


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
#import necessary libraries
import warnings
warnings.filterwarnings('ignore')
import itertools
import pandas as pd
from pandas import Series
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#load the data set and view the first five rows
df = pd.read_csv("zillow_data.csv")
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [3]:
df.City.value_counts()

New York            114
Los Angeles          95
Houston              88
San Antonio          48
Washington           45
Chicago              41
Austin               40
Springfield          40
Columbus             38
Las Vegas            38
Philadelphia         38
Phoenix              37
Kansas City          37
Dallas               35
Portland             34
San Diego            33
Louisville           32
Omaha                30
Denver               30
Oklahoma City        30
Arlington            29
Jacksonville         28
San Jose             27
Richmond             27
Fort Worth           26
Indianapolis         26
Lincoln              25
Charlotte            25
Madison              25
Franklin             25
                   ... 
Fort Loramie          1
Little Ferry          1
South Plainfield      1
Stanhope              1
Winthrop Harbor       1
Ottertail             1
Groveport             1
Andes                 1
Upper Chichester      1
Keewatin              1
Hutchins        

The data set features 14,723 zipcodes in 7,554 cities. For the purposes of this research we will look into the zipcodes in Houston, TX. Houston is one of the fastest growing cities in the US. Every day, about 250 people move to Houston, and with the population of 2,325,502 people in 2018, it is projected to overtake Chicago as the 3rd largest city in the country.

In [5]:
#select the data for Houston and view the first 5 rows
df = df.loc[df['City'] == 'Houston']
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
44,91685,77036,Houston,TX,Houston,Harris,45,120400.0,118700.0,117300.0,...,174900,174600,175000,176000,177200,177700,177700,179800,185100,189800
63,91726,77077,Houston,TX,Houston,Harris,64,177100.0,180000.0,182700.0,...,309600,310000,310800,311600,311800,311200,310500,311400,313800,315900
182,91744,77095,Houston,TX,Houston,Harris,183,131700.0,131300.0,131100.0,...,208900,210200,210700,211200,212300,213100,212800,214300,217500,219600
279,91731,77082,Houston,TX,Houston,Harris,280,101600.0,101300.0,100800.0,...,165800,166600,167600,168400,169000,169800,170800,171800,172500,172900


In [9]:
#view the last 5 rows
df.tail()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
8230,91681,77032,Houston,TX,Houston,Harris,8231,57100.0,57100.0,57000.0,...,104800,105600,106500,108200,111200,113600,115600,117400,118800,119200
8357,91743,77094,Houston,TX,Houston,Harris,8358,356300.0,357700.0,358800.0,...,375100,374400,374100,374700,375300,375000,373400,372800,373600,373900
8897,75034,38851,Houston,MS,NaN,Chickasaw,8898,NaN,NaN,NaN,...,74200,74500,74900,75300,76100,74400,72000,72000,73800,74000
11934,91699,77050,Houston,TX,Houston,Harris,11935,49200.0,49500.0,49700.0,...,104200,107000,109500,111200,110800,110900,112900,114900,115400,115200
14206,66105,19954,Houston,DE,Dover,Kent,14207,NaN,NaN,NaN,...,157000,156400,155700,154900,153200,152000,152500,153100,153200,153500


We can see that some of the lines present the data for Houston, MS and Houston, DE. Let's make sure we are only considering the data for Houston, TX.

In [6]:
#select the data for Houston, TX and view the first 5 rows
df = df.drop(df[(df['State'] != 'TX')].index)
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
44,91685,77036,Houston,TX,Houston,Harris,45,120400.0,118700.0,117300.0,...,174900,174600,175000,176000,177200,177700,177700,179800,185100,189800
63,91726,77077,Houston,TX,Houston,Harris,64,177100.0,180000.0,182700.0,...,309600,310000,310800,311600,311800,311200,310500,311400,313800,315900
182,91744,77095,Houston,TX,Houston,Harris,183,131700.0,131300.0,131100.0,...,208900,210200,210700,211200,212300,213100,212800,214300,217500,219600
279,91731,77082,Houston,TX,Houston,Harris,280,101600.0,101300.0,100800.0,...,165800,166600,167600,168400,169000,169800,170800,171800,172500,172900


In [7]:
#check the dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 5 to 11934
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 183.4+ KB


In [8]:
#check for NaN values
df.isna().sum()

RegionID      0
RegionName    0
City          0
State         0
Metro         0
CountyName    0
SizeRank      0
1996-04       0
1996-05       0
1996-06       0
1996-07       0
1996-08       0
1996-09       0
1996-10       0
1996-11       0
1996-12       0
1997-01       0
1997-02       0
1997-03       0
1997-04       0
1997-05       0
1997-06       0
1997-07       0
1997-08       0
1997-09       0
1997-10       0
1997-11       0
1997-12       0
1998-01       0
1998-02       0
             ..
2015-11       0
2015-12       0
2016-01       0
2016-02       0
2016-03       0
2016-04       0
2016-05       0
2016-06       0
2016-07       0
2016-08       0
2016-09       0
2016-10       0
2016-11       0
2016-12       0
2017-01       0
2017-02       0
2017-03       0
2017-04       0
2017-05       0
2017-06       0
2017-07       0
2017-08       0
2017-09       0
2017-10       0
2017-11       0
2017-12       0
2018-01       0
2018-02       0
2018-03       0
2018-04       0
Length: 272, dtype: int6

We now have a table featuring 86 zip codes in Houston, TX. The table does not contain NaN values are we are ready to reshape it from wide to long format.

<img src='HOU Zip.png'>

# Step 4: Reshape from Wide to Long Format

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

In [16]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [17]:
def melt_df(df):
    merged = []
    for zipcode in df.RegionName:
        melted = melt_data(df.loc[df['RegionName'] == zipcode])
        row = df.loc[df['RegionName'] == zipcode].iloc[:,:7]
        rows = pd.concat([row]*len(melted), ignore_index=True)
        merge = pd.concat([rows, melted.reset_index()], axis= 1)
        merged.append(merge)
    melted_df = pd.concat(merged)
    return melted_df

In [18]:
df = melt_df(df)

In [19]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,91733,77084,Houston,TX,Houston,Harris,6,1996-04-01,95000.0
1,91733,77084,Houston,TX,Houston,Harris,6,1996-05-01,95200.0
2,91733,77084,Houston,TX,Houston,Harris,6,1996-06-01,95400.0
3,91733,77084,Houston,TX,Houston,Harris,6,1996-07-01,95700.0
4,91733,77084,Houston,TX,Houston,Harris,6,1996-08-01,95900.0


# Step 2: Data Preprocessing

In [20]:
df.time = pd.to_datetime(df.time, format='%d/%m/%y')

In [21]:
df.set_index('time', inplace = True)

In [22]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,value
time,,,,,,,,
1996-04-01,91733,77084,Houston,TX,Houston,Harris,6,95000.0
1996-05-01,91733,77084,Houston,TX,Houston,Harris,6,95200.0
1996-06-01,91733,77084,Houston,TX,Houston,Harris,6,95400.0
1996-07-01,91733,77084,Houston,TX,Houston,Harris,6,95700.0
1996-08-01,91733,77084,Houston,TX,Houston,Harris,6,95900.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22790 entries, 1996-04-01 to 2018-04-01
Data columns (total 8 columns):
RegionID      22790 non-null int64
RegionName    22790 non-null int64
City          22790 non-null object
State         22790 non-null object
Metro         22790 non-null object
CountyName    22790 non-null object
SizeRank      22790 non-null int64
value         22790 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 1.6+ MB


# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results